In [2]:
import numpy as np
import cv2
import pickle
from matplotlib import pyplot as plt
%matplotllib inline
import os
import re
from sklearn.utils import shuffle
from  datetime import datetime

ERROR:root:Line magic function `%matplotllib` not found.


In [3]:
import keras
from keras.preprocessing import image, sequence
from keras.applications import VGG16
from keras.layers import Convolution2D, MaxPooling2D, Activation, Dense, Flatten, Reshape,ZeroPadding2D,Dropout
from keras.models import Sequential,Model
from keras.utils import np_utils

Using TensorFlow backend.


## Preprocessing 

In [4]:
# specifying number of categories in our data there is 2 dog and cat
no_of_classes = 2

In [5]:
datapath = '/my_data/PetImages/'
files = os.listdir(datapath)
# print files
paths = []
labels = []
for f in files:
    if f == '.DS_Store':
        continue
    ims = os.listdir(datapath + f)
    
    for px in ims:
        if px == '.DS_Store':
            continue
        if px.endswith(".jpg"):
            paths.append(datapath + f + '/' + px)
            labels.append(f)
        

In [6]:
# Shuffle the data to make training more robust
paths, labels = shuffle(paths, labels, random_state=0)

In [7]:
print(len(paths))

25000


In [8]:
img = []
img_labels = []

# I just have taken 10k images due to memory constraints

startTime = datetime.now()
for path,label in zip(paths[:10000],labels[:10000]):
    try:
        im = image.load_img(path, target_size=(224,224,3))
        im = image.img_to_array(im)
        img.append(im)
        img_labels.append(label)
    except:
        pass
print ("Time Elapsed: ",datetime.now()-startTime)

/usr/local/lib/python3.5/site-packages/PIL/TiffImagePlugin.py:692: UserWarning: Possibly corrupt EXIF data.  Expecting to read 262146 bytes but only got 0. Skipping tag 2
  " Skipping tag %s" % (size, len(data), tag))
/usr/local/lib/python3.5/site-packages/PIL/TiffImagePlugin.py:692: UserWarning: Possibly corrupt EXIF data.  Expecting to read 262151 bytes but only got 0. Skipping tag 56
  " Skipping tag %s" % (size, len(data), tag))
/usr/local/lib/python3.5/site-packages/PIL/TiffImagePlugin.py:709: UserWarning: Corrupt EXIF data.  Expecting to read 12 bytes but only got 0. 
  warnings.warn(str(msg))


Time Elapsed:  0:01:37.550256


In [9]:
len(img)

9998

In [10]:
uniques, ids = np.unique(img_labels, return_inverse=True)
labels = np_utils.to_categorical(ids, len(uniques))

In [11]:
with open( "/output/img_list.p", "wb" ) as pickle_f:
    pickle.dump(uniques, pickle_f )  

In [12]:
X = np.asarray(img)
y = np.asarray(labels)
print (X.shape,y.shape)

(9998, 224, 224, 3) (9998, 2)


In [13]:
X, y = shuffle(X, y, random_state=0)

In [14]:
split = int(0.7*X.shape[0])
X_train = X[:split]
y_train = y[:split]

X_val = X[split:]
y_val = y[split:]

print (X_train.shape,y_train.shape)

(6998, 224, 224, 3) (6998, 2)


## Model Architecturing

In [15]:
def VGG_16_fine_tune(weights_path=None):
    # Extract Bottleneck features
    vgg = VGG16(weights='imagenet', include_top=True, input_shape=(224,224,3))
    x = vgg.layers[-2].output
    x = Dropout(0.5)(x)
    x = Dense(no_of_classes, activation='softmax', name='prediction')(x)
    model = Model(inputs=vgg.input, outputs=x)
    
    # Set Bottom layers as non-trainable to reduce computations
    for l in model.layers[:-1]:
        l.trainable = False
    
    if weights_path:
        model.load_weights(weights_path)
    return model

In [16]:
model = VGG_16_fine_tune()

In [17]:
model.summary()
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

### Train the model

In [20]:
# I have run it for 2+10 epochs
startTime = datetime.now()
hist = model.fit(X_train, y_train,
                nb_epoch=1,
                shuffle=True,
                batch_size=100,
                validation_data=(X_val, y_val))
endTime = datetime.now()

/usr/local/lib/python3.5/site-packages/ipykernel_launcher.py:7: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  import sys


Train on 6998 samples, validate on 3000 samples
Epoch 1/1
6998/6998 [==============================] - 154s - loss: 0.1732 - acc: 0.9583 - val_loss: 0.0927 - val_acc: 0.9690


In [21]:
# save model and it's weights
model.save_weights("/output/model_weights.h5")

##  lets train for more iterations

In [22]:
hist2 =  model.fit(X_train, y_train,
                nb_epoch=5,
                shuffle=True,
                batch_size=100,
                validation_data=(X_val, y_val))
# save model and it's weights
model.save_weights("/output/model2_weights.h5")

/usr/local/lib/python3.5/site-packages/ipykernel_launcher.py:5: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  """


Train on 6998 samples, validate on 3000 samples
Epoch 1/5
6998/6998 [==============================] - 154s - loss: 0.1292 - acc: 0.9631 - val_loss: 0.1211 - val_acc: 0.9643
Epoch 2/5
6998/6998 [==============================] - 155s - loss: 0.1073 - acc: 0.9666 - val_loss: 0.0903 - val_acc: 0.9680
Epoch 3/5
6998/6998 [==============================] - 155s - loss: 0.1010 - acc: 0.9690 - val_loss: 0.0932 - val_acc: 0.9680
Epoch 4/5
6998/6998 [==============================] - 155s - loss: 0.1065 - acc: 0.9673 - val_loss: 0.1074 - val_acc: 0.9673
Epoch 5/5
6998/6998 [==============================] - 155s - loss: 0.0811 - acc: 0.9721 - val_loss: 0.1069 - val_acc: 0.9673


In [23]:
model_json = model.to_json()
with open("/output/model2.json", "w") as json_file:
    json_file.write(model_json)

### Weights can be downloaded from

https://www.floydhub.com/viewer/data/uDHLqkRSAS3tMpxpibxP4C/PYVnzvHEbhfSPHRtNznBii/model2_weights.h5
